# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [ ]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
# DONE: Import the necessary libs
# For example: 
from datetime import datetime
import json
import os
from typing import Dict, List, TypedDict, Union

from lib.agents import Agent
from lib.llm import LLM
from lib.messages import UserMessage, SystemMessage, ToolMessage, AIMessage, BaseMessage
from lib.tooling import tool
from lib.rag import RAG
from lib.state_machine import StateMachine, Step, EntryPoint, Termination, Resource
from lib.vector_db import VectorStore

import chromadb
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from tavily import TavilyClient

In [ ]:
# DONE: Load environment variables
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [ ]:
# DONE: Create retrieve_game tool
# It should use chroma client and collection you created
# chroma_client = chromadb.PersistentClient(path="chromadb")
# 
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - query: a question about game industry. 
#
#    You'll receive results as list. Each element contains:
#    - Platform: like Game Boy, Playstation 5, Xbox 360...)
#    - Name: Name of the Game
#    - YearOfRelease: Year when that game was released for that platform
#    - Description: Additional details about the game

chroma_client = chromadb.PersistentClient(path="chromadb")
collection = chroma_client.get_collection("udaplay")

@tool
def retrieve_game(query: str) :
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """

    print(f"Calling retrieve_game: query=\"{query}\"")

    results = collection.query(
        query_texts=[query],
        n_results=10
    )

    metadatas = results.get("metadatas", [[]])[0]

    result = [
        {
            "Platform": m.get("Platform"),
            "Name": m.get("Name"),
            "YearOfRelease": m.get("YearOfRelease"),
            "Description": m.get("Description")
        }
        for m in metadatas
    ]

    print(f"Returning: {json.dumps(result, indent=2)}")
    return result

#### Evaluate Retrieval Tool

In [ ]:
# DONE: Create evaluate_retrieval tool
# You might use an LLM as judge in this tool to evaluate the performance
# You need to prompt that LLM with something like:
# "Your task is to evaluate if the documents are enough to respond the query. "
# "Give a detailed explanation, so it's possible to take an action to accept it or not."
# Use EvaluationReport to parse the result
# Tool Docstring:
#    Based on the user's question and on the list of retrieved documents, 
#    it will analyze the usability of the documents to respond to that question. 
#    args: 
#    - question: original question from user
#    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
#    The result includes:
#    - useful: whether the documents are useful to answer the question
#    - description: description about the evaluation result

judge_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

class EvaluationResult(BaseModel):
    useful: bool = Field(..., description="Whether the documents are useful to answer the question.")
    description: str = Field(..., description="Explanation of why the documents are or are not useful.")

class EvaluationResultList(BaseModel):
    evaluations: List[EvaluationResult]

@tool
def evaluate_retrieval(question: str, retrieved_docs: list) -> Dict:
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 

    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database

    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """

    print(f"Calling evaluate_retrieval: question=\"{question}\", retrieved_docs={json.dumps(retrieved_docs, indent=2)}")

    response = judge_llm.invoke([
        SystemMessage(content="""
Your task is to evaluate if the documents are enough to respond the query.
Give a detailed explanation, so it's possible to take an action to accept it or not.
        """),
        UserMessage(content=f"""
The user asked:

### Question
{question}

### Retrieved Documents
{json.dumps(retrieved_docs, indent=2)}

Evaluate **each** document *individually*.

For each document:
1. Determine whether this single document is useful for answering the question.
2. Provide a short explanation.

Return the output in the following JSON structure:

{{
  "evaluations": [
    {{
      "useful": true/false,
      "description": "short explanation for doc #1"
    }},
    {{
      "useful": true/false,
      "description": "short explanation for doc #2"
    }},
    ...
  ]
}}
        """)
    ],
        response_format=EvaluationResultList
    )

    result = json.loads(response.content)

    print(f"Returning {json.dumps(result, indent=2)}")
    return result

#### Game Web Search Tool

In [ ]:
# DONE: Create game_web_search tool
# Please use Tavily client to search the web
# Tool Docstring:
#    Semantic search: Finds most results in the vector DB
#    args:
#    - question: a question about game industry. 
@tool
def game_web_search(query: str) -> Dict:
    """
    Search the web using Tavily API
    args:
    - question: a question about game industry. 
    """

    print(f"Calling game_web_search: query=\"{query}\"")

    client = TavilyClient(api_key=TAVILY_API_KEY)

    # Perform the search
    search_result = client.search(
        query=query,
        search_depth="advanced",
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )

    # Format the results
    result = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }

    print(f"Returning {json.dumps(result, indent=2)}")
    return result

### Agent

In [ ]:
# DONE: Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed

# Define agent state
class State(TypedDict):
    messages: List[BaseMessage]
    question: str
    documents: List[str]
    doc_search_results: List
    doc_evaluation_results: List
    doc_filtered_results: List
    web_search_results: Dict
    result: str

In [ ]:
def doc_search(state: State) -> State:
    # Look up results
    results = retrieve_game(state["question"])
    state["doc_search_results"] = results
    return state

def doc_evaluate(state: State) -> State:
    # Evaluate results and discard irrelevant results
    evaluation_results = evaluate_retrieval(state["question"], state["doc_search_results"])
    state["doc_evaluation_results"] = evaluation_results["evaluations"]
    return state

def doc_filter(state: State) -> State:
    doc_search_results = state["doc_search_results"]
    doc_evaluation_results = state["doc_evaluation_results"]
    state["doc_filtered_results"] = [
        doc_search_result
        for doc_search_result, evaluation_result in zip(doc_search_results, doc_evaluation_results)
        if evaluation_result["useful"]
    ]
    return state

def doc_answer(state: State) -> State:
    output_llm = LLM(
        model="gpt-4o-mini",
        temperature=0.3,
    )
    response = output_llm.invoke([
        SystemMessage(content=f"""
Your task is to output the answer to a question, given retrieved documents containing the relevant information.
        """),
        UserMessage(content=f"""
The user asked:

### Question
{state["question"]}

### Retrieved Documents
{json.dumps(state["doc_filtered_results"], indent=2)}
        """)
    ])
    state["result"] = response.content
    return state

def web_search(state: State) -> State:
    state["web_search_results"] = game_web_search(state["question"])
    return state

def web_answer(state: State) -> State:
    output_llm = LLM(
        model="gpt-4o-mini",
        temperature=0.3,
    )
    response = output_llm.invoke([
        SystemMessage(content=f"""
Your task is to output the answer to a question, given web search results. Include citations to URLs in your answer.
        """),
        UserMessage(content=f"""
The user asked:

### Question
{state["question"]}

### Web Search Results
{json.dumps(state["web_search_results"], indent=2)}
        """)
    ])
    state["result"] = response.content
    return state

In [ ]:
workflow = StateMachine(State)

In [ ]:
# Create steps
entry = EntryPoint()
doc_search_step = Step[State]("doc_search", doc_search)
doc_evaluate_step = Step[State]("doc_evaluate", doc_evaluate)
doc_filter_step = Step[State]("doc_filter", doc_filter)
doc_answer_step = Step[State]("doc_answer", doc_answer)
web_search_step = Step[State]("web_search", web_search)
web_answer_step = Step[State]("web_answer", web_answer)
termination = Termination()

workflow.add_steps(
    [
        entry, 
        doc_search_step, 
        doc_evaluate_step, 
        doc_answer_step, 
        doc_filter_step,
        web_search_step,
        web_answer_step,
        termination
    ]
)

In [ ]:
# Add transitions
workflow.connect(entry, doc_search_step)
workflow.connect(doc_search_step, doc_evaluate_step)
workflow.connect(doc_evaluate_step, doc_filter_step)

def check_doc_results(state) -> Union[Step[State], str]:
    if state["doc_filtered_results"]:
        return doc_answer_step
    return web_search_step

workflow.connect(doc_filter_step, [doc_answer_step, web_search_step], check_doc_results)
workflow.connect(doc_answer_step, termination)
workflow.connect(web_search_step, web_answer_step)
workflow.connect(web_answer_step, termination)

In [ ]:
# DONE: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

In [ ]:
def invoke_agent(question: str) -> str:
    initial_state: State = {
        "question": question
    }
    run_object = workflow.run(initial_state)
    return run_object.get_final_state()["result"]

print(invoke_agent("When Pokémon Gold and Silver was released?"))

In [ ]:
print(invoke_agent("Which one was the first 3D platformer Mario game?"))

In [ ]:
print(invoke_agent("Was Mortal Kombat X realeased for Playstation 5?"))

### (Optional) Advanced

In [ ]:
# TODO: Update your agent with long-term memory
# TODO: Convert the agent to be a state machine, with the tools being pre-defined nodes